## Задание 5.1

Набор данных тут: https://github.com/sismetanin/rureviews, также есть в папке [Data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). Те, кто предпочитает работать с английским языком, могут использовать набор данных `sms_spam`.

In [203]:
!pip install gdown


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [204]:
!gdown 1j-DhO_XD5EqzOqVSR4Wz1kNzhLmpeTkZ -O ./data/women-clothing-accessories.csv

Downloading...
From: https://drive.google.com/uc?id=1j-DhO_XD5EqzOqVSR4Wz1kNzhLmpeTkZ
To: /Users/artyomfadeyev/GitHub/ITMO-PROBLEMS/Machine-Learning/V semester/data/women-clothing-accessories.csv
100%|██████████████████████████████████████| 21.8M/21.8M [00:08<00:00, 2.58MB/s]


In [205]:
import pandas as pd

accessories = pd.read_csv('./data/women-clothing-accessories.csv', sep='\t')

In [206]:
accessories.head()

,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative
1,"Товар отдали другому человеку, я не получила п...",negative
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative
3,"товар не пришел, продавец продлил защиту без м...",negative
4,"Кофточка голая синтетика, носить не возможно.",negative


Применим полученные навыки и решим задачу анализа тональности отзывов. 

Нужно повторить весь пайплайн от сырых текстов до получения обученной модели.

Обязательные шаги предобработки:
1. токенизация
2. приведение к нижнему регистру
3. удаление стоп-слов
4. лемматизация
5. векторизация (с настройкой гиперпараметров)
6. построение модели
7. оценка качества модели

Обязательно использование векторайзеров:
1. мешок n-грамм (диапазон для n подбирайте самостоятельно, запрещено использовать только униграммы).
2. tf-idf ((диапазон для n подбирайте самостоятельно, также нужно подбирать гиперпараметры max_df, min_df, max_features)
3. символьные n-граммы (диапазон для n подбирайте самостоятельно)

В качестве классификатора нужно использовать наивный байесовский классификатор. 

Для сравнения векторайзеров между собой используйте precision, recall, f1-score и accuracy. Для этого сформируйте датафрейм, в котором в строках будут разные векторайзеры, а в столбцах разные метрики качества, а в  ячейках будут значения этих метрик для соответсвующих векторайзеров.

In [207]:
!pip install nltk


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [208]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

def tokenize(text):
    return nltk.word_tokenize(text)

def to_lower(tokens):
    return [token.lower() for token in tokens]

stop_words = set(nltk.corpus.stopwords.words('russian'))
def remove_stopwords(tokens):
    return [
        token
        for token in tokens
        if token not in stop_words
    ]

def lemmatize(tokens):
    lemmatizer = nltk.stem.WordNetLemmatizer()
    return [
        lemmatizer.lemmatize(token)
        for token in tokens
    ]

def remove_non_words(tokens):
    return [
        token
        for token in tokens
        if token.isalpha()
    ]


def preprocess(text):
    tokens = tokenize(text)
    tokens = to_lower(tokens)
    tokens = remove_stopwords(tokens)
    tokens = lemmatize(tokens)
    tokens = remove_non_words(tokens)
    return ' '.join(tokens)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/artyomfadeyev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/artyomfadeyev/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/artyomfadeyev/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/artyomfadeyev/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [209]:
accessories.review = accessories.review.apply(preprocess)
accessories.head()

,review,sentiment
0,качество плохое пошив ужасный горловина напере...,negative
1,товар отдали другому человеку получила посылку...,negative
2,ужасная синтетика тонкая общего представленной...,negative
3,товар пришел продавец продлил защиту моего сог...,negative
4,кофточка голая синтетика носить возможно,negative


In [210]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    accessories.review,
    accessories.sentiment,
    test_size=0.2,
    random_state=42
)

In [211]:
def generate_multiple_ngram_ranges(min_n, max_n):
    for lower in range(min_n, max_n + 1):
        for upper in range(lower, max_n + 1):
            yield lower, upper

In [212]:
import time
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
import warnings

warnings.filterwarnings('always')


def naive_bayes_classifier(X_train, X_test, y_train, y_test, vectorizer, name):
    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)
    classifier = MultinomialNB()
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    print(f'\n{time.ctime()}: {name}\n')
    return report

In [213]:
from sklearn.feature_extraction.text import CountVectorizer


def create_count_vectorizers_words():
    for ngram_range in generate_multiple_ngram_ranges(2, 6):
        yield f'CountVectorizer ngram_range={ngram_range} analyzer=word', CountVectorizer(
                ngram_range=ngram_range,
                analyzer='word',
            )

report_data = []
def make_report(X_train, X_test, y_train, y_test, vectorizers):
    for name, vectorizer in vectorizers:
        try:
            report = naive_bayes_classifier(
                X_train, X_test, y_train, y_test, vectorizer, name
            )
        except ValueError as e:
            print(f'{name} failed: {e}')
            continue

        report_data.append((name, report))

In [214]:
make_report(X_train, X_test, y_train, y_test, create_count_vectorizers_words())


Tue Nov 22 13:11:17 2022: CountVectorizer ngram_range=(2, 2) analyzer=word


Tue Nov 22 13:11:21 2022: CountVectorizer ngram_range=(2, 3) analyzer=word


Tue Nov 22 13:11:27 2022: CountVectorizer ngram_range=(2, 4) analyzer=word


Tue Nov 22 13:11:36 2022: CountVectorizer ngram_range=(2, 5) analyzer=word


Tue Nov 22 13:11:46 2022: CountVectorizer ngram_range=(2, 6) analyzer=word


Tue Nov 22 13:11:48 2022: CountVectorizer ngram_range=(3, 3) analyzer=word


Tue Nov 22 13:11:53 2022: CountVectorizer ngram_range=(3, 4) analyzer=word


Tue Nov 22 13:12:00 2022: CountVectorizer ngram_range=(3, 5) analyzer=word


Tue Nov 22 13:12:08 2022: CountVectorizer ngram_range=(3, 6) analyzer=word


Tue Nov 22 13:12:11 2022: CountVectorizer ngram_range=(4, 4) analyzer=word


Tue Nov 22 13:12:16 2022: CountVectorizer ngram_range=(4, 5) analyzer=word


Tue Nov 22 13:12:22 2022: CountVectorizer ngram_range=(4, 6) analyzer=word


Tue Nov 22 13:12:25 2022: CountVectorizer ngram_range=(5, 5) analyzer=word


In [215]:
from sklearn.feature_extraction.text import TfidfVectorizer


def create_tfidf_vectorizers():
    for max_df in [0.2, 0.3, 1.0]:
        for min_df in [0, 0.01, 0.025]:
            for max_features in [1000, 5000, 10000, 50000]:
                for ngram_range in generate_multiple_ngram_ranges(2, 3):
                    yield f'TfidfVectorizer max_df={max_df} min_df={min_df} max_features={max_features} ngram_range={ngram_range}', TfidfVectorizer(
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        ngram_range=ngram_range,
                    )

In [216]:
make_report(X_train, X_test, y_train, y_test, create_tfidf_vectorizers())


Tue Nov 22 13:12:34 2022: TfidfVectorizer max_df=0.2 min_df=0 max_features=1000 ngram_range=(2, 2)


Tue Nov 22 13:12:39 2022: TfidfVectorizer max_df=0.2 min_df=0 max_features=1000 ngram_range=(2, 3)


Tue Nov 22 13:12:42 2022: TfidfVectorizer max_df=0.2 min_df=0 max_features=1000 ngram_range=(3, 3)


Tue Nov 22 13:12:44 2022: TfidfVectorizer max_df=0.2 min_df=0 max_features=5000 ngram_range=(2, 2)


Tue Nov 22 13:12:48 2022: TfidfVectorizer max_df=0.2 min_df=0 max_features=5000 ngram_range=(2, 3)


Tue Nov 22 13:12:51 2022: TfidfVectorizer max_df=0.2 min_df=0 max_features=5000 ngram_range=(3, 3)


Tue Nov 22 13:12:53 2022: TfidfVectorizer max_df=0.2 min_df=0 max_features=10000 ngram_range=(2, 2)


Tue Nov 22 13:12:58 2022: TfidfVectorizer max_df=0.2 min_df=0 max_features=10000 ngram_range=(2, 3)


Tue Nov 22 13:13:01 2022: TfidfVectorizer max_df=0.2 min_df=0 max_features=10000 ngram_range=(3, 3)


Tue Nov 22 13:13:04 2022: TfidfVectorizer max_df=0.2 min_df=0 max_features=50000 ngram_

In [217]:
def create_count_vectorizers_letters():
    for ngram_range in generate_multiple_ngram_ranges(2, 6):
        yield f'CountVectorizer ngram_range={ngram_range} analyzer=char', CountVectorizer(
                ngram_range=ngram_range,
                analyzer='char',
            )

In [218]:
make_report(X_train, X_test, y_train, y_test, create_count_vectorizers_letters())


Tue Nov 22 13:18:09 2022: CountVectorizer ngram_range=(2, 2) analyzer=char


Tue Nov 22 13:18:14 2022: CountVectorizer ngram_range=(2, 3) analyzer=char


Tue Nov 22 13:18:24 2022: CountVectorizer ngram_range=(2, 4) analyzer=char


Tue Nov 22 13:18:40 2022: CountVectorizer ngram_range=(2, 5) analyzer=char


Tue Nov 22 13:19:01 2022: CountVectorizer ngram_range=(2, 6) analyzer=char


Tue Nov 22 13:19:05 2022: CountVectorizer ngram_range=(3, 3) analyzer=char


Tue Nov 22 13:19:11 2022: CountVectorizer ngram_range=(3, 4) analyzer=char


Tue Nov 22 13:19:23 2022: CountVectorizer ngram_range=(3, 5) analyzer=char


Tue Nov 22 13:19:40 2022: CountVectorizer ngram_range=(3, 6) analyzer=char


Tue Nov 22 13:19:43 2022: CountVectorizer ngram_range=(4, 4) analyzer=char


Tue Nov 22 13:19:51 2022: CountVectorizer ngram_range=(4, 5) analyzer=char


Tue Nov 22 13:20:05 2022: CountVectorizer ngram_range=(4, 6) analyzer=char


Tue Nov 22 13:20:09 2022: CountVectorizer ngram_range=(5, 5) analyzer=char


In [219]:
import typing


def compare(report_data: typing.List[typing.Tuple[str, str]]):
    return pd.DataFrame([
            {'name': name, **report['weighted avg']}
            for name, report in report_data
        ])


combined = compare(report_data)

In [220]:
combined.sort_values(by='f1-score', ascending=False).head(15)

,name,precision,recall,f1-score,support
113,"CountVectorizer ngram_range=(6, 6) analyzer=char",0.709594,0.705722,0.706322,18000
112,"CountVectorizer ngram_range=(5, 6) analyzer=char",0.709258,0.704667,0.705145,18000
110,"CountVectorizer ngram_range=(4, 6) analyzer=char",0.708820,0.702722,0.703493,18000
107,"CountVectorizer ngram_range=(3, 6) analyzer=char",0.707118,0.700111,0.701035,18000
111,"CountVectorizer ngram_range=(5, 5) analyzer=char",0.705747,0.699944,0.700654,18000
103,"CountVectorizer ngram_range=(2, 6) analyzer=char",0.707112,0.699000,0.700048,18000
109,"CountVectorizer ngram_range=(4, 5) analyzer=char",0.705569,0.698500,0.699375,18000
106,"CountVectorizer ngram_range=(3, 5) analyzer=char",0.702779,0.694500,0.695590,18000
108,"CountVectorizer ngram_range=(4, 4) analyzer=char",0.700184,0.692333,0.693363,18000
102,"CountVectorizer ngram_range=(2, 5) analyzer=char",0.701149,0.691667,0.692925,18000


In [221]:
combined.sort_values(by='precision', ascending=False).head(15)

,name,precision,recall,f1-score,support
113,"CountVectorizer ngram_range=(6, 6) analyzer=char",0.709594,0.705722,0.706322,18000
112,"CountVectorizer ngram_range=(5, 6) analyzer=char",0.709258,0.704667,0.705145,18000
110,"CountVectorizer ngram_range=(4, 6) analyzer=char",0.708820,0.702722,0.703493,18000
107,"CountVectorizer ngram_range=(3, 6) analyzer=char",0.707118,0.700111,0.701035,18000
103,"CountVectorizer ngram_range=(2, 6) analyzer=char",0.707112,0.699000,0.700048,18000
111,"CountVectorizer ngram_range=(5, 5) analyzer=char",0.705747,0.699944,0.700654,18000
109,"CountVectorizer ngram_range=(4, 5) analyzer=char",0.705569,0.698500,0.699375,18000
106,"CountVectorizer ngram_range=(3, 5) analyzer=char",0.702779,0.694500,0.695590,18000
102,"CountVectorizer ngram_range=(2, 5) analyzer=char",0.701149,0.691667,0.692925,18000
108,"CountVectorizer ngram_range=(4, 4) analyzer=char",0.700184,0.692333,0.693363,18000


In [222]:
combined.sort_values(by='recall', ascending=False).head(15)

,name,precision,recall,f1-score,support
113,"CountVectorizer ngram_range=(6, 6) analyzer=char",0.709594,0.705722,0.706322,18000
112,"CountVectorizer ngram_range=(5, 6) analyzer=char",0.709258,0.704667,0.705145,18000
110,"CountVectorizer ngram_range=(4, 6) analyzer=char",0.708820,0.702722,0.703493,18000
107,"CountVectorizer ngram_range=(3, 6) analyzer=char",0.707118,0.700111,0.701035,18000
111,"CountVectorizer ngram_range=(5, 5) analyzer=char",0.705747,0.699944,0.700654,18000
103,"CountVectorizer ngram_range=(2, 6) analyzer=char",0.707112,0.699000,0.700048,18000
109,"CountVectorizer ngram_range=(4, 5) analyzer=char",0.705569,0.698500,0.699375,18000
106,"CountVectorizer ngram_range=(3, 5) analyzer=char",0.702779,0.694500,0.695590,18000
108,"CountVectorizer ngram_range=(4, 4) analyzer=char",0.700184,0.692333,0.693363,18000
102,"CountVectorizer ngram_range=(2, 5) analyzer=char",0.701149,0.691667,0.692925,18000


In [223]:
combined.sample(10)

,name,precision,recall,f1-score,support
104,"CountVectorizer ngram_range=(3, 3) analyzer=char",0.685008,0.674000,0.675682,18000
10,"CountVectorizer ngram_range=(4, 5) analyzer=word",0.524333,0.402667,0.329883,18000
78,TfidfVectorizer max_df=1.0 min_df=0 max_featur...,0.643260,0.637944,0.638625,18000
73,TfidfVectorizer max_df=1.0 min_df=0 max_featur...,0.527410,0.418944,0.366263,18000
7,"CountVectorizer ngram_range=(3, 5) analyzer=word",0.562971,0.534278,0.519073,18000
82,TfidfVectorizer max_df=1.0 min_df=0 max_featur...,0.559140,0.507611,0.491500,18000
92,TfidfVectorizer max_df=1.0 min_df=0.025 max_fe...,0.199355,0.295222,0.165981,18000
110,"CountVectorizer ngram_range=(4, 6) analyzer=char",0.708820,0.702722,0.703493,18000
108,"CountVectorizer ngram_range=(4, 4) analyzer=char",0.700184,0.692333,0.693363,18000
67,TfidfVectorizer max_df=0.3 min_df=0.025 max_fe...,0.199355,0.295222,0.165981,18000


Как вы видим, лучше всего себя показал `CountVectorizer` по символам с `ngram_range=(6, 6)`.

## Задание 5.2 Регулярные выражения

Регулярные выражения - способ поиска и анализа строк. Например, можно понять, какие даты в наборе строк представлены в формате DD/MM/YYYY, а какие - в других форматах. 

Или бывает, например, что перед работой с текстом, надо почистить его от своеобразного мусора: упоминаний пользователей, url и так далее.

Навык полезный, давайте в нём тоже потренируемся.

Для работы с регулярными выражениями есть библиотека **re**

In [224]:
import re

В регулярных выражениях, кроме привычных символов-букв, есть специальные символы:
* **?а** - ноль или один символ **а**
* **+а** - один или более символов **а**
* **\*а** - ноль или более символов **а** (не путать с +)
* **.** - любое количество любого символа

Пример:
Выражению \*a?b. соответствуют последовательности a, ab, abc, aa, aac НО НЕ abb!

Рассмотрим подробно несколько наиболее полезных функций:

### findall
возвращает список всех найденных непересекающихся совпадений.

Регулярное выражение **ab+c.**: 
* **a** - просто символ **a**
* **b+** - один или более символов **b**
* **c** - просто символ **c**
* **.** - любой символ


In [225]:
result = re.findall('ab+c.', 'abcdefghijkabcabcxabc')
print(result)

['abcd', 'abca']


Вопрос на внимательность: почему нет abcx?
- Множества непересекающиеся

**Задание**: вернуть список первых двух букв каждого слова в строке, состоящей из нескольких слов.

In [226]:
text = 'The quick brown fox jumps over the lazy dog.'
two_first_letters = re.findall(r'\w\w', text)
if len(two_first_letters) <= 1:
    two_first_letters = []

print(two_first_letters)

['Th', 'qu', 'ic', 'br', 'ow', 'fo', 'ju', 'mp', 'ov', 'er', 'th', 'la', 'zy', 'do']


### split
разделяет строку по заданному шаблону


In [227]:
result = re.split(',', 'itsy, bitsy, teenie, weenie')
print(result)

['itsy', ' bitsy', ' teenie', ' weenie']


можно указать максимальное количество разбиений

In [228]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit=2)
print(result)

['itsy', ' bitsy', ' teenie, weenie']


**Задание**: разбейте строку, состоящую из нескольких предложений, по точкам, но не более чем на 3 предложения.

In [229]:
multiple_sentences_text = 'aboba. bebra. aboba bebra. lol. kek.'
re.split(r'\.', multiple_sentences_text, maxsplit=2)

['aboba', ' bebra', ' aboba bebra. lol. kek.']

### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [230]:
result = re.sub('a', 'b', 'abcabc')
print (result)

bbcbbc


**Задание**: напишите регулярное выражение, которое позволит заменить все цифры в строке на "DIG".

In [231]:
digits_text = '322 is really good number'
re.sub(r'\d', 'DIG', digits_text)

'DIGDIGDIG is really good number'

**Задание**: напишите  регулярное выражение, которое позволит убрать url из строки.

In [232]:
# Регулярная довольно тупая, в ней довольно легко можно найти лазейку, но нормальная
# Будет очень большой и сложной

url_text = 'https://aboba.com is my awesome web site'
re.sub(r'https?://(www\.)?[\w\d.:?/=]+', '', url_text).strip()

'is my awesome web site'

### compile
компилирует регулярное выражение в отдельный объект

In [233]:
# Пример: построение списка всех слов строки:
prog = re.compile('[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

<>:2: DeprecationWarning: invalid escape sequence \-
<>:2: DeprecationWarning: invalid escape sequence \-
/var/folders/cz/rsh24k2d5sq_37dlsjbd5mb40000gn/T/ipykernel_47646/1061151323.py:2: DeprecationWarning: invalid escape sequence \-
  prog = re.compile('[А-Яа-яё\-]+')


['Слова', 'Да', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё']

**Задание**: для выбранной строки постройте список слов, которые длиннее трех символов.

In [234]:
more_than_three_letters = re.compile(r'\w{4,}')
more_than_three_letters.findall('aboba bebra aboba bebra lol kek')

['aboba', 'bebra', 'aboba', 'bebra']

**Задание**: вернуть список доменов (@gmail.com) из списка адресов электронной почты:

```
abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz
```

In [235]:
domains = "abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz"
domains_pattern = re.compile(r'@[\w\d.]+')
domains_pattern.findall(domains)

['@gmail.com', '@test.in', '@analyticsvidhya.com', '@rest.biz']